In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torchaudio
from torchaudio import transforms
from torch.utils.data import Dataset
from torchmetrics.text.wer import WordErrorRate
from torchmetrics.text.cer import CharErrorRate
from torch.autograd import Variable

import pandas as pd
import os
import re
from tqdm.notebook import tqdm
import gc
from numpy.testing import assert_almost_equal
import math
import numpy as np

from conformer_model import ConformerEncoder, LSTMDecoder
from conformer_utils import *

In [2]:
DEVICE = "cuda:0"
# TRAIN_BS = 16
# TEST_BS = 10
# EPOCHS = 50
validation_transform = transforms.MelSpectrogram(sample_rate=16000, 
                                                 n_mels=80, 
                                                 hop_length=160)

In [3]:
# checkpoint_path = "F:\RK-IITK\SURGE-23\Code_practice\Project_Begins_21_May\model\conf_s_lj_best_wer.pt"
# checkpoint_path="F:\RK-IITK\SURGE-23\Code_practice\Project_Begins_21_May\model\conf_s_libri_460_best_wer (1).pt"
checkpoint_path="F:\RK-IITK\SURGE-23\Code_practice\Dataset\Long_Audio_Data_Annotators\PMS\conformer_S_hindi_full_Mar19.pt"

char_dict = load_char_dict(checkpoint_path, DEVICE)
print(char_dict)

char_list = list(char_dict.keys())
print(char_list)

{' ': 0, 'ँ': 1, 'ं': 2, 'ः': 3, 'अ': 4, 'आ': 5, 'इ': 6, 'ई': 7, 'उ': 8, 'ऊ': 9, 'ऋ': 10, 'ए': 11, 'ऐ': 12, 'ऑ': 13, 'ओ': 14, 'औ': 15, 'क': 16, 'ख': 17, 'ग': 18, 'घ': 19, 'च': 20, 'छ': 21, 'ज': 22, 'झ': 23, 'ञ': 24, 'ट': 25, 'ठ': 26, 'ड': 27, 'ढ': 28, 'ण': 29, 'त': 30, 'थ': 31, 'द': 32, 'ध': 33, 'न': 34, 'प': 35, 'फ': 36, 'ब': 37, 'भ': 38, 'म': 39, 'य': 40, 'र': 41, 'ल': 42, 'व': 43, 'श': 44, 'ष': 45, 'स': 46, 'ह': 47, 'ा': 48, 'ि': 49, 'ी': 50, 'ु': 51, 'ू': 52, 'ृ': 53, 'े': 54, 'ै': 55, 'ॉ': 56, 'ो': 57, 'ौ': 58, '्': 59, 'ज़': 60, 'ड़': 61, 'ढ़': 62, 'फ़': 63}
[' ', 'ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ए', 'ऐ', 'ऑ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'े', 'ै', 'ॉ', 'ो', 'ौ', '्', 'ज़', 'ड़', 'ढ़', 'फ़']


In [4]:
encoder_params = {
    "d_input": 80,
    "d_model": 144,
    "num_layers": 16,
    "conv_kernel_size": 32,
    "dropout": 0.1,
    "num_heads": 4
}

decoder_params = {
    "d_encoder": 144,
    "d_decoder": 320,
    "num_layers": 1,
    "num_classes":len(char_list)+1
}

In [5]:
encoder = ConformerEncoder(
                      d_input=encoder_params['d_input'],
                      d_model=encoder_params['d_model'],
                      num_layers=encoder_params['num_layers'],
                      conv_kernel_size=encoder_params['conv_kernel_size'], 
                      dropout=encoder_params['dropout'],
                      num_heads=encoder_params['num_heads']
                    )
  
decoder = LSTMDecoder(
                  d_encoder=decoder_params['d_encoder'], 
                  d_decoder=decoder_params['d_decoder'], 
                  num_layers=decoder_params['num_layers'],
                    num_classes= decoder_params['num_classes'])

encoder = encoder.to(DEVICE)
decoder = decoder.to(DEVICE)

In [6]:
load_model_inference(encoder, decoder, checkpoint_path, DEVICE)

(83, tensor(0.0175, device='cuda:0'), tensor(20.8077, device='cuda:0'))

In [7]:
def ConformerForward(encoder, decoder, char_decoder, spectrograms, mask, char_list, gpu=True):
    ''' Evaluate model on test dataset. '''

    encoder.eval()
    decoder.eval()

    # Move to GPU
    if gpu:
        spectrograms = spectrograms.cuda()

        mask = mask.cuda()

    with torch.no_grad():
        outputs, attention_cem = encoder(spectrograms, mask)
        outputs, decoder_cem = decoder(outputs)
 
        soft_cem = F.softmax(outputs, dim=-1)


        inds, uncollapsed_inds = char_decoder(outputs.detach())
        
        uncollapsed_predictions = []
        
        for sample1 in uncollapsed_inds:
            uncollapsed_predictions.append(int_to_text_uncollapse(sample1, len(char_list), char_list))

        collapse_predictions = []
        final_predictions = []
        for sample in inds:
            collapse_predictions.append(int_to_text_collapse(sample, len(char_list), char_list))
            final_predictions.append(int_to_text_final(sample, len(char_list), char_list))

    return attention_cem, decoder_cem, soft_cem, final_predictions, uncollapsed_predictions,outputs

In [8]:
char_decoder = GreedyCharacterDecoder().to(DEVICE)

In [9]:
def audio_process(waveform):
    mel_spec = validation_transform(waveform) # (channels, n_mels, time)
    mel_spec = mel_spec.squeeze(0).transpose(0,1)  # (time, n_mels)
    mel_spec_len = ((mel_spec.shape[0] - 1) // 2 - 1) // 2

    input_list = []
    input_list.append(mel_spec)
    input_tens = nn.utils.rnn.pad_sequence(input_list, batch_first=True)

    input_len_list = []
    input_len_list.append(mel_spec_len)

    mask = torch.ones(input_tens.shape[0], input_tens.shape[1], input_tens.shape[1])
    for i, l in enumerate(input_len_list):
        mask[i, :, :l] = 0
    return input_tens, mask.bool()

In [10]:
# reading the tesv file
# data_file_new="F:\RK-IITK\SURGE-23\Code_practice\Dataset\Long_Audio_Data_Annotators\PMS\metadata_final.tsv"
data_file_new="F:\RK-IITK\SURGE-23\Code_practice\Dataset\Long_Audio_Data_Annotators\\VerifiedTextFiles"
# read txt files from the folder and make a single pd dataframe with all the files name and texts in the files
def read_txt_files(data_file_new):
    df=pd.DataFrame(columns=['file_name','text'])
    for file in tqdm(os.listdir(data_file_new)):
        if file.endswith(".txt"):
            with open(os.path.join(data_file_new, file), 'r',encoding="utf-8") as f:
                text=f.read()
                # remove extra spaces
                text=re.sub(' +', ' ', text)
                df=df.append({'file_name':file,'text':text},ignore_index=True)
    return df

    
df_hindi=read_txt_files(data_file_new)
df_hindi
    
# df_hindi=pd.read_csv(data_file_new,sep='\t',header=None)
# df_hindi

  0%|          | 0/45 [00:00<?, ?it/s]

C:\Users\Legion\AppData\Local\Temp\ipykernel_16192\1632185370.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({'file_name':file,'text':text},ignore_index=True)
C:\Users\Legion\AppData\Local\Temp\ipykernel_16192\1632185370.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({'file_name':file,'text':text},ignore_index=True)
C:\Users\Legion\AppData\Local\Temp\ipykernel_16192\1632185370.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({'file_name':file,'text':text},ignore_index=True)
C:\Users\Legion\AppData\Local\Temp\ipykernel_16192\1632185370.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instea

,file_name,text
0,10janA.txt,नमस्कार दोपहर समाचार में आपका स्वागत है भारत न...
1,10_mar_noon.txt,नमस्कार दोपहर समाचार में आपका स्वागत है प्रधान...
2,11janA.txt,नमस्कार दोपहर समाचार में आपका स्वागत है प्रधान...
3,11_mar_noon.txt,नमस्कार दोपहर समाचार में आपका स्वागत है प्रधान...
4,12janA.txt,नमस्कार दोपहर समाचार में आपका स्वागत है प्रधान...
5,12_mar_noon.txt,नमस्कार दोपहर समाचार में आपका स्वागत है प्रधान...
6,13janA.txt,नमस्कार दोपहर समाचार में आपका स्वागत है प्रधान...
7,13_mar_noon.txt,नमस्कार दोपहर समाचार में आपका स्वागत है भारत न...
8,14janA.txt,नमस्कार दोपहर समाचार में आपका स्वागत है राष्ट्...
9,14_mar_noon.txt,नमस्कार दोपहर समाचार में आपका स्वागत है संसद क...


In [11]:
# df_hindi['26_feb_23.txt','text']
# get text of 26_feb_23.txt file from df_hindi
# df_hindi[df_hindi['file_name']=='10_04_2023.txt']['text']


In [12]:
# ref=[]
# april_transcript=''

# for i in range(185):
#     april_transcript+=df_hindi[1][i]

# ref.append(april_transcript)

# august_transcript=''

# for i in range(185,186+155):
#     august_transcript+=df_hindi[1][i]
    


# december_transcript=''

# for i in range(186+155,186+156+162):
#     december_transcript+=df_hindi[1][i]
    

    
# july_transcript=''

# for i in range(186+156+162,186+156+162+153):
#     july_transcript+=df_hindi[1][i]
    
# ref.append(july_transcript)
# ref.append(august_transcript)
# ref.append(december_transcript)
# ref

In [13]:
# ref = []
pred = []

In [14]:
df_hindi

,file_name,text
0,10janA.txt,नमस्कार दोपहर समाचार में आपका स्वागत है भारत न...
1,10_mar_noon.txt,नमस्कार दोपहर समाचार में आपका स्वागत है प्रधान...
2,11janA.txt,नमस्कार दोपहर समाचार में आपका स्वागत है प्रधान...
3,11_mar_noon.txt,नमस्कार दोपहर समाचार में आपका स्वागत है प्रधान...
4,12janA.txt,नमस्कार दोपहर समाचार में आपका स्वागत है प्रधान...
5,12_mar_noon.txt,नमस्कार दोपहर समाचार में आपका स्वागत है प्रधान...
6,13janA.txt,नमस्कार दोपहर समाचार में आपका स्वागत है प्रधान...
7,13_mar_noon.txt,नमस्कार दोपहर समाचार में आपका स्वागत है भारत न...
8,14janA.txt,नमस्कार दोपहर समाचार में आपका स्वागत है राष्ट्...
9,14_mar_noon.txt,नमस्कार दोपहर समाचार में आपका स्वागत है संसद क...


In [15]:
# def beam_search_decoder (output, beam_width=3):
#     probabilities = F.softmax(output, dim=2)
#     blank_idx = probabilities.size(2) - 1
#     sequence_length = output.size(1)

#     # Initialize the beam search candidates with the first timestep
#     topk_probs, topk_chars = torch. topk(probabilities[0, 0], k=beam_width)
#     candidates = []

#     for i in range(beam_width):
#         candidates.append({
#             'sequence': [topk_chars[i].item()],
#             'probability': topk_probs[i].item(),
#         })

#     # Iterate over the remaining timesteps
#     for t in range(1, sequence_length):
#         new_candidates = []

#         for candidate in candidates:
#             sequence = candidate['sequence']
#             probability = candidate ['probability']

#     # Get the probabilities and characters for the current timestep
#             timestep_probs = probabilities[0, t]
#             topk_probs, topk_chars = torch. topk(timestep_probs, k=beam_width)

#             for i in range(beam_width):
#                 new_probability = probability + topk_probs[i].item()
#                 new_sequence = sequence + [topk_chars[i].item()]
#                 new_candidates. append({
#                 'sequence': new_sequence,
#                 'probability': new_probability,
#                 })

#     # Keep the top-k candidates with the highest probabilities

#         candidates = sorted(new_candidates, key=lambda x: x['probability'], reverse=True)[:beam_width]

#     # Return the sequence with the highest probability

#     best_sequence_uncollapsed = candidates[0]['sequence']

#     tensor_sequence = torch. tensor(best_sequence_uncollapsed)

#     best_sequence_collapsed = indices = torch.unique_consecutive(tensor_sequence, dim=-1)
#     best_sequence_uncollapsed = [c for c in best_sequence_collapsed if c != blank_idx]

#     return tensor_sequence,best_sequence_uncollapsed,best_sequence_collapsed, tensor_sequence.tolist()


In [16]:
from torchaudio.models.decoder import ctc_decoder

LM_WEIGHT = 3.23
WORD_SCORE = -0.26

beam_search_decoder = ctc_decoder(
    lexicon="F:\RK-IITK\SURGE-23\Code_practice\Project_Begins_21_May\speech_conformer_inference\speech_conformer_inference\VAD_tests\Beam_search_files\\lexicon.txt",
    tokens="F:\RK-IITK\SURGE-23\Code_practice\Project_Begins_21_May\speech_conformer_inference\speech_conformer_inference\VAD_tests\Beam_search_files\\tokens.txt",
    lm=None,
    nbest=3,
    beam_size=1500,
    lm_weight=LM_WEIGHT,
    word_score=WORD_SCORE,
)

In [28]:
waveforms=[]
# wav_files=('PM_2022_april','PM_2022_july','PM_2022_august','PM_2022_december')
# wav_files=('PM_2022_april','PM_2022_august')
# for i in tqdm(range(len(wav_files))):
# read df_hindi file, open audios with same file name
file_path="F:\RK-IITK\SURGE-23\Code_practice\Dataset\Long_Audio_Data_Annotators\\allWav"
    # file_path="F:\RK-IITK\SURGE-23\Code_practice\Dataset\Long_Audio_Data_Annotators\Audio\\"+wav_files[i]+".wav"
    # file_path = df.iloc[i,0]
    # ref.append(df.iloc[i,1])


# make new column in df_hindi named waveforms



    # open audio files, do the below processing and then make a new colimn in df_hindi and put waveform corresponding to same file name in first column with txt
    # file_path="F:\RK-IITK\SURGE-23\Code_practice\Dataset\Long_Audio_Data_Annotators\audios\audios"
for file in tqdm(os.listdir(file_path)):
        # print(file)
        # open only wav files
    if file.endswith(".wav"):
        waveform, sample_rate = torchaudio.load(os.path.join(file_path, file), normalize = True)
        channel_dim = waveform.shape[0]
        if channel_dim > 1:
            waveform = torch.mean(waveform, 0, keepdim=True)
        if sample_rate!=16000:
            waveform = transforms.Resample(orig_freq=sample_rate, new_freq=16000)(waveform)
    
        # insert in df_hindi in the row with the same file name
        txt_file=file.split('.')[0]+'.txt'
        # df_hindi.loc[df_hindi['file_name']==txt_file]['waveform'] = waveform
        waveforms.append({'file_name':txt_file,'waveform':waveform})
        
    
   
#     # waveform, sample_rate = torchaudio.load(file_path, normalize = True)
#     channel_dim = waveform.shape[0]
#     if channel_dim > 1:
#         waveform = torch.mean(waveform, 0, keepdim=True)
#     if sample_rate!=16000:
#        waveform = transforms.Resample(orig_freq=sample_rate, new_freq=16000)(waveform)
#     waveforms.append(waveform)
    
print(waveforms)
  
    

  0%|          | 0/45 [00:00<?, ?it/s]

[{'file_name': '10janA.txt', 'waveform': tensor([[ 0.0000,  0.0000,  0.0000,  ..., -0.0458, -0.0383, -0.0342]])}, {'file_name': '10_mar_noon.txt', 'waveform': tensor([[ 0.0000,  0.0000,  0.0000,  ..., -0.0562, -0.0611, -0.0562]])}, {'file_name': '11janA.txt', 'waveform': tensor([[ 0.0000,  0.0000,  0.0000,  ..., -0.0003, -0.0003, -0.0003]])}, {'file_name': '11_mar_noon.txt', 'waveform': tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0948, 0.0973, 0.1031]])}, {'file_name': '12janA.txt', 'waveform': tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0005, 0.0005, 0.0005]])}, {'file_name': '12_mar_noon.txt', 'waveform': tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0135, 0.0143, 0.0126]])}, {'file_name': '13janA.txt', 'waveform': tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0289, 0.0284, 0.0330]])}, {'file_name': '13_mar_noon.txt', 'waveform': tensor([[ 0.0000,  0.0000,  0.0000,  ..., -0.1315, -0.1629, -0.1539]])}, {'file_name': '14janA.txt', 'waveform': tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0211, 0.0199, 0.02

In [18]:
# specifying duration of the chunks and the strides
sample_rate = 16000
chunk_size = 12
# stride_size = 1

chunk_samples = int(chunk_size * sample_rate)
# stride_samples = int(stride_size * sample_rate)

print(chunk_samples)
# print(stride_samples)

192000


In [20]:
# test=waveforms[0]['waveform'][0][0:chunk_samples]
# input_tens, mask = audio_process(test)
# _, _, _, final_predictions, uncollapsed_predictions,outputs = ConformerForward(encoder,
#                                                         decoder, char_decoder, input_tens, mask, 
#                                                         char_list)

# tensor_sequence,uncollapsed,collapsed,_=beam_search_decoder(outputs,beam_width=3)
# print(tensor_sequence)
# print(uncollapsed)
# print(collapsed)

TypeError: __call__() got an unexpected keyword argument 'beam_width'

In [ ]:
def give_pred(uncollapsed_indices):
    pred=[]
  
    indices = torch.unique_consecutive(uncollapsed_indices, dim=-1)
    indices.tolist()
    for sample in indices:
       pred.append(int_to_text_final(sample, len(char_list), char_list))
       
    return pred[0]

In [ ]:
pred

[]

In [ ]:
# # waveform=waveforms[0]
# print(waveform['waveform'])
# test=waveform['waveform'][0][0:chunk_samples]
# input_tens, mask = audio_process(test)
# _, _, _, final_predictions, uncollapsed_predictions,outputs = ConformerForward(encoder,
#                                                         decoder, char_decoder, input_tens, mask, 
#                                                         char_list)     
# uncollapsed_predictions

In [ ]:
print(len(uncollapsed_predictions[0]))
uncollapsed_indices=(torch.argmax(outputs,dim=-1))
print(uncollapsed_indices.size())

# for i in range(0,len(uncollapsed_indices[0])):
#     print(f"{i}: {uncollapsed_indices[0][i]} {uncollapsed_predictions[0][i]}")
    
# i=0

# while(i<len(uncollapsed_indices[0])):
#     temp=0
#     while(uncollapsed_indices[0][i]==64 or uncollapsed_indices[0][i]==0):
#         i=i+1
#         temp=temp+1
#     if(temp>5):
#         print(temp)
#     i=i+1

i=len(uncollapsed_indices[0])-1

while(i>=0):
    
    
    length=0
    while(i>=0):
        if(uncollapsed_indices[0][i]==0 or uncollapsed_indices[0][i]==64):
            length=length+1
            i=i-1
        else:
            break
        
    if(length>10):
        print(f'end: {i} length: {length}')
        break
    i=i-1
print(uncollapsed_indices[0][0:int(i+length/2)])
#     temp=0
#     while(uncollapsed_indices[0][i]==64 or uncollapsed_indices[0][i]==0):
#         i=i+1
#         temp=temp+1

299
torch.Size([1, 299])
end: 79 length: 219
tensor([64, 64, 64, 34, 64, 39, 46, 46, 59, 59, 16, 48, 41, 41, 64, 64, 64, 64,
        64, 64, 64, 64, 64, 64,  0,  0, 32, 57, 64, 35, 64, 47, 64, 41, 64, 64,
         0, 64, 46, 64, 39, 48, 64, 20, 48, 48, 41, 64, 64,  0, 39, 54,  2, 64,
        64,  0, 64,  5, 35, 64, 64, 16, 48, 64, 64,  0,  0, 46, 59, 43, 48, 48,
        18, 30, 30, 64,  0,  0, 47, 55, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64,
        64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64,
        64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64,
        64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64,
        64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64,
        64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64,
        64, 64, 64, 64, 64, 64, 64, 64], device='cuda:0')


In [19]:
ref=[]
for i in range(len(df_hindi)):
    ref.append(df_hindi['text'].values[i])

# ref

In [20]:
chunk_samples=16000*15

In [21]:
import datetime
import time


In [29]:
# waveform=waveforms[0]
df_time_stamps=pd.DataFrame(columns=['file_name','duration','start_time','end_time','text'])

# ref=[]
pred=[]
time_stamps=[]
chunk_transcriptions_list=[]
# j=0
waveforms_new=waveforms[-1:]
for waveform in tqdm(waveforms_new):
    # if(j==1) : break/
    # j=j+1
    file=waveform['file_name']
    print(file)
    # text_wave=df_hindi[df_hindi['file_name']==file]['text'].values[0]
    # print(text_wave)
    # ref.append(text_wave)
    waveform=waveform['waveform']
    chunk_transcriptions=[]
    wave_time_stamps= []
    start=0
    transcription=''
    while(1):
        # print(f"start: {start}")
        if(start+chunk_samples<waveform.size()[1]): test=waveform[0][start:start+chunk_samples]
        else : 
            test=waveform[0][start:]
            input_tens, mask = audio_process(test)
            _, _, ok, final_predictions, uncollapsed_predictions,outputs = ConformerForward(encoder,decoder, char_decoder, input_tens, mask,char_list)
            # uncollapsed_indices,_,_,_=beam_search_decoder(outputs,beam_width=3)
            # final=give_pred(uncollapsed_indices.unsqueeze(0))
            beam_search_result = beam_search_decoder(ok.cpu())
            beam_search_transcript = " ".join(beam_search_result[0][0].words).strip()
                        
            transcription=transcription+" "+beam_search_transcript
            break 

        input_tens, mask = audio_process(test)
        _, _, ok, final_predictions, uncollapsed_predictions,outputs = ConformerForward(encoder,decoder, char_decoder, input_tens, mask,char_list)
        
        # uncollapsed_indices,_,_,_=beam_search_decoder(outputs,beam_width=3)
        

        
        uncollapsed_indices=(torch.argmax(outputs.detach(),dim=-1))
        # print(uncollapsed_indices[0].size())
        i=len(uncollapsed_indices[0])-1
        # print(i)
        while(i>=0):
            length=0
            while(i>=0):
                if(uncollapsed_indices[0][i]==0 or uncollapsed_indices[0][i]==64):
                    length=length+1
                    i=i-1
                else:
                    break
            if(length>9):
                uncollapsed_indices=uncollapsed_indices[0][0:int(i+length/2)]
                break
            i=i-1
        uncol_len=len(uncollapsed_indices)
        # print(uncol_len)
        # start_time=datetime.timedelta(seconds=start/16000)
        # end_time=datetime.timedelta(seconds=(start+uncol_len*640)/16000)
        
        new_start=start+uncol_len*640
        if(uncol_len<=1): 
            start=new_start
            continue
        # print(ok.shape)
        ok=ok[0:,0:int(i+length/2),0:]
        beam_search_result = beam_search_decoder(ok.cpu())
        beam_search_transcript = " ".join(beam_search_result[0][0].words).strip()
        
        # chunk_pred=give_pred(uncollapsed_indices.unsqueeze(0))
        chunk_pred=beam_search_transcript
        chunk_transcriptions.append(chunk_pred)
        start_time=start
        end_time=start+uncol_len*640
        # duration=end_time-start_time
        # start_time=time.strftime('%H:%M:%S', time.gmtime(start_time/16000))
        # end_time=time.strftime('%H:%M:%S', time.gmtime((end_time)/16000))
        # duration=time.strftime('%H:%M:%S', time.gmtime(duration/16000))
        # wave_time_stamps.append([start_time,end_time])
        # df_time_stamps=df_time_stamps.append({'file_name':file,'duration':duration,'start_time':start_time,'end_time':end_time,'text':chunk_pred},ignore_index=True)
        transcription=transcription+' '+(chunk_pred)
        start=new_start
    pred.append(transcription)
    chunk_transcriptions_list.append(chunk_transcriptions)
    time_stamps.append(wave_time_stamps)
    
    
    

  0%|          | 0/1 [00:00<?, ?it/s]

PM_2022_may.txt


In [25]:
for i in range(0,5):
    print(f"ref: {ref[i]}")
    print(f"pred: {pred[i]}")
    # print("\n")
    
# print(ref[:5])
# print(pred)

ref: नमस्कार दोपहर समाचार में आपका स्वागत है भारत ने कोलकाता में जी ट्वंटी वित्तीय समावेशन की बैठक में सभी के लिए न्यायसंगत और समान विकास का आह्वान किया राष्ट्रपति द्रौपदी मुर्मु आज प्रवासी भारतीय सम्मान प्रदान करेगी उत्तराखण्ड सरकार ने जोशीमठ में असुरक्षित भवनों को गिराने का निर्णय लिया प्रधानमंत्री नरेन्द्र मोदी बृहस्पतिवार को कनार्टक के हुबली में राष्ट्रीय युवा महोत्सव का उद्घाटन करेंगे द कश्मीर फाइल सहित पांच भारतीय फिल्म ऑस्कर पुरस्कारों की पहली सूची में शामिल आज विश्व हिन्दी दिवस मनाया जा रहा है गुवाहाटी में भारत के साथ पहले एक दिवसीय अंतरराष्ट्रीय क्रिकेट मैच में श्रीलंका का टॉस जीतकर गेंदबाजी का फैसला दोपहर समाचार के साथ मैं मुकेश कुमार वित्तीय समावेशन के लिए वैश्विक भागीदारी पर जी ट्वंटी कार्यसमूह की पहली बैठक का पूर्ण सत्र आज सुबह कोलकाता में शुरू हुआ उद्घाटन भाषण में जनजातीय कार्य मंत्री अर्जुन मुंडा ने सभी के लिए न्यायपूर्ण और समान विकास की आवश्यकता पर बल दिया वसुधैव कुटुम्बकम के भारत के दृष्टिकोण को दोहराते हुए उन्होंने कहा कि हम एक मानव केंद्रित वैश्वीकरण का प्रयास करते ह

In [26]:
len(chunk_transcriptions)
df_time_stamps
wer=WordErrorRate()
print(wer(ref[:5],pred))

tensor(0.5242)


In [33]:
# df_time_stamps.to_csv('time_stamps.tsv',index=False)
# i want to save df_time_stamps as tsv file

df_time_stamps.to_csv('time_stamps.tsv',sep='\t',index=False)






In [93]:
file='10_04_2023.txt'
df_hindi[df_hindi['file_name']==file]['text'].values[0]

'नमस्कार दोपहर समाचार मे आप का स्वागत है उच् चतम न् यायालय ने केंद्र की अग्निपथ योजना को सही ठहराने के दिल्ली उच् च न् यायालय के फैसले को चुनौती देने वाली याचिकाएं खारिज कीं गृह मंत्री अमित शाह आज दोपहर बाद अरुणाचल प्रदेश में वाइब्रेंट विलेजेस कार्यक्रम का शुभारंभ करेंगे कोविड प्रबंधन की तैयारियों की समीक्षा के लिए राष्ट्रव्यापी मॉक ड्रिल सभी सरकारी और निजी अस्पतालों में चल रही है उपराष्ट्रपति जगदीप धनखड़ ने नई दिल्ली में विश्व होम्योपैथी दिवस पर वैज्ञानिक सम्मेलन का उद्घाटन किया और आईपीएल क्रिकेट में आज शाम बेंगलूरू में रॉयल चैलेंजर्स बैंगलोर का मुकाबला लखनऊ सुपर जायंट्स से होगा दोपहर समाचार के साथ मैं कनक लता उच् चतम न्यायालय ने केंद्र सरकार की अग्निपथ योजना को सही ठहराने वाले दिल्ली उच् च न् यायालय के निर्णय को चुनौती देने वाली दो याचिकाओं को आज खारिज कर दिया सर्वोच् च न् यायालय ने अग्निपथ की वैधता को सही ठहराने वाले दिल्ली उच् च न् यायालय के फैसले को बरकरार रखा है एक रिपोर्ट प्रधान न्यायाधीश डी वाई चंद्रचूड़ की अध्यक्षता वाली पीठ ने कहा कि अग्निपथ योजना की शुरुआत से पहले रक्षा बलों क

In [95]:
time_stamps

[[['00:00:00', '00:00:09'],
  ['00:00:09', '00:00:23'],
  ['00:00:23', '00:00:38'],
  ['00:00:38', '00:00:52'],
  ['00:00:52', '00:01:01'],
  ['00:01:01', '00:01:12'],
  ['00:01:12', '00:01:22'],
  ['00:01:22', '00:01:36'],
  ['00:01:36', '00:01:47'],
  ['00:01:47', '00:01:58'],
  ['00:01:58', '00:02:12'],
  ['00:02:12', '00:02:26'],
  ['00:02:26', '00:02:39'],
  ['00:02:39', '00:02:50'],
  ['00:02:50', '00:03:04'],
  ['00:03:04', '00:03:10'],
  ['00:03:10', '00:03:10'],
  ['00:03:11', '00:03:20'],
  ['00:03:20', '00:03:35'],
  ['00:03:36', '00:03:36'],
  ['00:03:37', '00:03:37'],
  ['00:03:37', '00:03:37'],
  ['00:03:40', '00:03:55'],
  ['00:03:55', '00:04:09'],
  ['00:04:09', '00:04:22'],
  ['00:04:22', '00:04:36'],
  ['00:04:36', '00:04:42'],
  ['00:04:42', '00:04:53'],
  ['00:04:53', '00:05:05'],
  ['00:05:05', '00:05:15'],
  ['00:05:15', '00:05:27'],
  ['00:05:27', '00:05:36'],
  ['00:05:36', '00:05:50'],
  ['00:05:50', '00:06:00'],
  ['00:06:00', '00:06:12'],
  ['00:06:12', '00:0

In [74]:
for j in range(len(chunk_transcriptions_list)):
    chunk_transcriptions=chunk_transcriptions_list[j]
    for i in range(len(chunk_transcriptions)):
        print(f"{(time_stamps[j][i])}: {chunk_transcriptions[i]}")
    

['00:00:00', '00:00:09']: नमस्कार दोपहर समाचार में आपका स्वागत है
['00:00:09', '00:00:23']: उच्चतम न्यायालय ने केंद्र की अग्निपथ योजना को सही ठहराने के दिल्ली उच्च न्यायालय के फैसली को चुनौती देने वाली याचिकाएं खारिज की
['00:00:23', '00:00:38']: गृह मंत्री अमित शाह आज दोपहर बाद अरुणाचल प्रदेश में वायबंड विलेजे कार्यक्रम का शुभारंभ करेंगे कोवड प्रबंधन की तैयारियों की समीक्षा के लिए राष्ट्रव्यापी मॉक्रन सभी सरकारी और निजी अस्पतालों में चल रही है
['00:00:38', '00:00:52']: उपराष्ट्रपति जगगीत धर्घर ने नई दिल्ली में विश्व होमोपथी दिवस पर वैज्ञानिक सम्मेलन का उद्घाटन किया और आईपीएल क्रिकेट में आज शाम बंगलुरु में रॉयल चैलेंजर्स बैंगलुरु का मुकाबला लखनऊ सुपर जांड से होगा
['00:00:52', '00:01:01']: दोपहर समाचार के साथ मैं कनक लता
['00:01:01', '00:01:12']: उच्चतम न्यायालय ने केंद्र सरकार की अग्निपथ योजना को सही ठहराने वाले दिल्लीय उच्च न्यायालय के निर्णय को चुनौती देने वाली दो याचिकाओं को आज खारिज कर दिया
['00:01:12', '00:01:22']: सर्वोच्च न्यायालय ने अग्निपथ की वैधिता को सही ठहराने वाले दिल्ली उच

In [96]:
# print(ref[0])
pred

[' नमस्कार दोपहर समाचार में आपका स्वागत है उच्चतम न्यायालय ने केंद्र की अग्निपथ योजना को सही ठहराने के दिल्ली उच्च न्यायालय के फैसली को चुनौती देने वाली याचिकाएं खारिज की गृह मंत्री अमित शाह आज दोपहर बाद अरुणाचल प्रदेश में वायबंड विलेजे कार्यक्रम का शुभारंभ करेंगे कोवड प्रबंधन की तैयारियों की समीक्षा के लिए राष्ट्रव्यापी मॉक्रन सभी सरकारी और निजी अस्पतालों में चल रही है उपराष्ट्रपति जगगीत धर्घर ने नई दिल्ली में विश्व होमोपथी दिवस पर वैज्ञानिक सम्मेलन का उद्घाटन किया और आईपीएल क्रिकेट में आज शाम बंगलुरु में रॉयल चैलेंजर्स बैंगलुरु का मुकाबला लखनऊ सुपर जांड से होगा दोपहर समाचार के साथ मैं कनक लता उच्चतम न्यायालय ने केंद्र सरकार की अग्निपथ योजना को सही ठहराने वाले दिल्लीय उच्च न्यायालय के निर्णय को चुनौती देने वाली दो याचिकाओं को आज खारिज कर दिया सर्वोच्च न्यायालय ने अग्निपथ की वैधिता को सही ठहराने वाले दिल्ली उच्च न्यायालय के फैसले को बरखरार रखा है एक रिपोर्ट प्रधान न्यायाधीश दीवाई चंद्र चूड़ की अध्यक्षता वाली पीठ ने कहा कि अग्निपथ योजना की शुरुआत से पहले रक्षाबलों के लिए भर्तीयहूं शारीरि

In [97]:
ref

['नमस्कार दोपहर समाचार मे आप का स्वागत है उच् चतम न् यायालय ने केंद्र की अग्निपथ योजना को सही ठहराने के दिल्ली उच् च न् यायालय के फैसले को चुनौती देने वाली याचिकाएं खारिज कीं गृह मंत्री अमित शाह आज दोपहर बाद अरुणाचल प्रदेश में वाइब्रेंट विलेजेस कार्यक्रम का शुभारंभ करेंगे कोविड प्रबंधन की तैयारियों की समीक्षा के लिए राष्ट्रव्यापी मॉक ड्रिल सभी सरकारी और निजी अस्पतालों में चल रही है उपराष्ट्रपति जगदीप धनखड़ ने नई दिल्ली में विश्व होम्योपैथी दिवस पर वैज्ञानिक सम्मेलन का उद्घाटन किया और आईपीएल क्रिकेट में आज शाम बेंगलूरू में रॉयल चैलेंजर्स बैंगलोर का मुकाबला लखनऊ सुपर जायंट्स से होगा दोपहर समाचार के साथ मैं कनक लता उच् चतम न्यायालय ने केंद्र सरकार की अग्निपथ योजना को सही ठहराने वाले दिल्ली उच् च न् यायालय के निर्णय को चुनौती देने वाली दो याचिकाओं को आज खारिज कर दिया सर्वोच् च न् यायालय ने अग्निपथ की वैधता को सही ठहराने वाले दिल्ली उच् च न् यायालय के फैसले को बरकरार रखा है एक रिपोर्ट प्रधान न्यायाधीश डी वाई चंद्रचूड़ की अध्यक्षता वाली पीठ ने कहा कि अग्निपथ योजना की शुरुआत से पहले रक्षा बलों 

In [25]:
wer=WordErrorRate()
# print(wer(ref,pred))
for i in range(len(pred)):
    print(f"{i}: {wer(ref[i],pred[i])}")
    # print("REFERENCE :")
    # print(len(ref[i]))
    # print("PRED :")
    # print(len(pred[i]))

0: 0.35169699788093567
1: 0.2617010474205017
2: 0.3178994953632355
3: 0.3393820524215698
4: 0.3867276906967163
5: 0.3996468484401703
6: 0.3105320334434509
7: 0.2989158630371094
8: 0.23961980640888214
9: 0.6589200496673584
10: 0.31674209237098694
11: 0.24786709249019623
12: 0.30343136191368103
13: 0.304736852645874
14: 0.3195619583129883
15: 0.4553714692592621
16: 0.3144104778766632
17: 0.31053104996681213
18: 0.32440873980522156
19: 0.26833978295326233
20: 0.3684483766555786
21: 0.26844972372055054
22: 0.2951258718967438
23: 0.33164042234420776
24: 1.0905719995498657
25: 0.2897959053516388
26: 0.2703920602798462
27: 0.3169916570186615
28: 0.3328092098236084
29: 0.263182669878006
30: 0.2744464874267578
31: 0.42848220467567444
32: 0.31676360964775085
33: 0.22861357033252716
34: 0.251080185174942
35: 0.3516709506511688
36: 0.21641790866851807
37: 0.3360897898674011
38: 0.4420555830001831
39: 0.2568047344684601
40: 0.18597964942455292
41: 0.1760939210653305
42: 0.17084860801696777
43: 0.16

In [45]:
# get last 5 entries of ref
ref[:-5]

['नमस्कार दोपहर समाचार में आपका स्वागत है भारत ने कोलकाता में जी ट्वंटी वित्तीय समावेशन की बैठक में सभी के लिए न्यायसंगत और समान विकास का आह्वान किया राष्ट्रपति द्रौपदी मुर्मु आज प्रवासी भारतीय सम्मान प्रदान करेगी उत्तराखण्ड सरकार ने जोशीमठ में असुरक्षित भवनों को गिराने का निर्णय लिया प्रधानमंत्री नरेन्द्र मोदी बृहस्पतिवार को कनार्टक के हुबली में राष्ट्रीय युवा महोत्सव का उद्घाटन करेंगे द कश्मीर फाइल सहित पांच भारतीय फिल्म ऑस्कर पुरस्कारों की पहली सूची में शामिल आज विश्व हिन्दी दिवस मनाया जा रहा है गुवाहाटी में भारत के साथ पहले एक दिवसीय अंतरराष्ट्रीय क्रिकेट मैच में श्रीलंका का टॉस जीतकर गेंदबाजी का फैसला दोपहर समाचार के साथ मैं मुकेश कुमार वित्तीय समावेशन के लिए वैश्विक भागीदारी पर जी ट्वंटी कार्यसमूह की पहली बैठक का पूर्ण सत्र आज सुबह कोलकाता में शुरू हुआ उद्घाटन भाषण में जनजातीय कार्य मंत्री अर्जुन मुंडा ने सभी के लिए न्यायपूर्ण और समान विकास की आवश्यकता पर बल दिया वसुधैव कुटुम्बकम के भारत के दृष्टिकोण को दोहराते हुए उन्होंने कहा कि हम एक मानव केंद्रित वैश्वीकरण का प्रयास करते हैं 

In [39]:
ref[-1:]

['मेरे प्यारे देशवासियो नमस्कार आज फिर एक बार मन की बात के माध्यम से आप सभी कोटि कोटि मेरे परिवारजनों से मिलने का अवसर मिला है मन की बात में आप सबका स्वागत है कुछ दिन पहले देश ने एक ऐसी उपलब्धि हासिल की है जो हम सभी को प्रेरणा देती है भारत के सामर्थ्य के प्रति एक नया विश्वास जगाती है आप लोग क्रिकेट के मैदान पर टीम इंडिया के किसी बैट्समैन की सेन्चुरी सुन कर खुश होते होंगे लेकिन भारत ने एक और मैदान में सेन्चुरी लगाई है और वो बहुत विशेष है इस महीने पाँच तारीख को देश में यूनिकॉर्न की संख्या सौ के आँकड़े तक पहुँच गई है और आपको तो पता ही है एक यूनिकॉर्न यानी कम से कम साढ़े सात हज़ार करोड़ रूपए का स्टार्ट अप इन यूनिकॉर्नस् का कुल वैल्यूऐशन तीन सौ तीस बिलियन डॉलर यानी पच्चीस लाख करोड़ रुपयों से भी ज्यादा है निश्चित रूप से ये बात हर भारतीय के लिए गर्व करने वाली बात है आपको यह जानकर भी हैरानी होगी कि हमारे कुल यूनिकॉर्न में से चवालीस फॉर्टी फोर पिछले साल बने थे इतना ही नहीं इस वर्ष के तीन चार महीने में ही चौदह और नए यूनिकॉर्न बन गए इसका मतलब यह हुआ कि ग्लोबल पैन्डेमिक के इस दौर में भी हमारे स्टार्ट अ

In [40]:
wer=WordErrorRate()
print(wer(ref[-1:],pred[0]))

tensor(0.1780)


In [46]:
wer=WordErrorRate()
print(wer(ref[:-5],pred[:-5]))

tensor(0.3367)


In [43]:
wer=WordErrorRate()
print(wer(ref[-5:],pred[-5:]))

tensor(0.1768)


In [77]:
cer = CharErrorRate()
print(cer(ref, pred))

tensor(0.0789)
